# Super Charging Objects in the Pipeline

## PowerShell Conference Europe 2025 
<img src="images/flag.jpg" style="width:20%;height:20%;" alt="Swedish flag">

### Jeff Hicks

<div style = "background-color:rgb(226, 220, 220); padding-bottom: 25px;padding-top:10px;padding-left: 10px; padding-right: 10px; width:60%">
<img src="images/hicks.png" alt="Jeff Hicks" align="left" vertical-align="top" style="padding-right: 10px ;padding-bottom: 25px;"/>

- 35-year IT Professional
- 19-year Microsoft MVP
- PowerShell Author 
- PowerShell Teacher
- https://jdhitsolutions.github.io
  
### ![MVP Logo](images/mvp-logo.png)

![Sponsors](images/psconfeu-sponsors.png)

## For Your Consideration

- How will your object be consumed?
  - Will it pass to another cmdlet?
  - Viewed on the screen?
- Could it be serialized? 
  - How will *that* be consumed?
- Do you need to make your work in an interactive session more efficient?
- What would *add value* for the user with __minimal__ effort on their part?

## Aliases

PowerShell can be verbose or obscure. What are you working with?

In [ ]:
Get-ChildItem c:\temp -File | Select-Object Name,
@{Name = 'Size'; Expression = { $_.length } },
@{Name = 'Modified'; Expression = { $_.LastWriteTime } } -first 5

Use `Update-TypeData` to extend what an object looks like.

- Applies to all objects of the same type
- Add aliases, properties, and methods
- Only persistent for the session

In [ ]:
$params = @{
    TypeName   = 'System.IO.FileInfo'
    MemberType = 'AliasProperty'
    MemberName = 'Size'
    Value      = 'Length'
    Force      = $True
}
Update-TypeData @params

$params.MemberName = 'Modified'
$params.Value = 'LastWriteTime'
Update-TypeData @params

In [ ]:
#this might be useful if serializing
$params.MemberType = 'NoteProperty'
$params.MemberName = 'Computername'
$params.Value = [System.Environment]::MachineName
Update-TypeData @params

In [ ]:
#this is much easier to type and read
Get-ChildItem c:\temp -File |
Select-Object Directory, Name, Size, Modified, ComputerName -first 5 |
Format-Table

## Script Properties

In [ ]:
Get-ChildItem c:\temp -File | Where-Object size -GT 500 |
Select-Object Name, Modified,
@{Name = 'ModifiedAge'; Expression = {New-TimeSpan -Start $_.LastWriteTime -End (Get-Date)}},
@{Name = 'SizeKB'; Expression = { $_.size / 1kb } }

In [ ]:
#Define a calculated script ScriptProperty for ModifiedAge
$params = @{
    TypeName   = 'System.IO.FileInfo'
    MemberType = 'ScriptProperty'
    MemberName = 'ModifiedAge'
    Value      = { New-TimeSpan -Start $this.LastWriteTime -End (Get-Date) }
    Force      = $True
}
Update-TypeData @params

In [ ]:
#create a second for SizeKB
$params.MemberName = 'SizeKB'
$params.value = { $this.Length / 1KB }
Update-TypeData @params

In [ ]:
#use them
Get-ChildItem c:\temp -File | Where-Object size -GT 500 |
Select-Object Name, Size, SizeKB, Modified, ModifiedAge, Computername -First 7 | Format-Table

In [ ]:
#script property code should run quickly but can be rich as you need
$test = {
    #test on the extension without the period
    If ($this.Extension) {
        Switch -regex ($this.Extension.Substring(1)) {
            '^ps(d)?1(xml)?' { 'PowerShell' }
            '^(bmp|jp(e)?g|png|gif|tiff)' { 'Image' }
            '^(mp3|mp4|m4v)' { 'Media' }
            '^(xml|json|csv|yml|yaml)' { 'Data' }
            '^(md|pdf|doc(x)?|htm(l)?|txt)' { 'Document' }
            '^(zip|tar|gz|bz2|7z)' { 'Archive' }
            '^(exe|dll|bat|cmd|com|pdb)' { 'System' }
            default { 'File' }
        }
    }
    else {
        'NULL'
    }
}

$params = @{
    TypeName   = 'System.IO.FileInfo'
    MemberType = 'ScriptProperty'
    MemberName = 'Category'
    Value      = $test
    Force      = $True
}
Update-TypeData @params

In [ ]:
Get-ChildItem c:\temp -File | Group-Object category

## Property Sets

In [ ]:
Get-ChildItem c:\temp -File |
Select-Object FullName, Size, CreationTime, Modified, ModifiedAge -first 7 | Format-Table

- A property set is a defined collection of properties
- Can be referenced by name
- Defined in a .ps1xml file
- Use https://github.com/jdhitsolutions/PSTypeExtensionTools

```powershell
$paramHash = @{
    Name       = 'AgeInfo'
    TypeName   = 'System.IO.FileInfo'
    Properties = 'FullName','Size','CreationTime','Modified','ModifiedAge'
    FilePath   = '.\jhFileinfo.types.ps1xml'
}
New-PSPropertySet @paramHash
```

```xml
<Types>
  <Type>
    <Name>System.IO.FileInfo</Name>
    <Members>
      <PropertySet>
        <Name>AgeInfo</Name>
        <ReferencedProperties>
          <Name>FullName</Name>
          <Name>Size</Name>
          <Name>CreationTime</Name>
          <Name>Modified</Name>
          <Name>ModifiedAge</Name>
        </ReferencedProperties>
      </PropertySet>
    </Members>
  </Type>
</Types>
```

In [ ]:
Update-TypeData -AppendPath .\jhFileinfo.types.ps1xml

In [ ]:
Get-ChildItem -File | Get-Member -MemberType PropertySet

In [ ]:
Get-ChildItem c:\temp -File | Select-Object AgeInfo -first 5 | Format-Table

## Script Methods

- When you want and object to do something
- Do something *to* the object
- You don't want to use a function

In [ ]:
# avoid using parameters
# Notice $this and not $_
$zip = {
    Param([string]$Destination = $this.Directory)
    $target = Join-Path -Path $Destination -ChildPath "$($this.baseName).zip"
    $paramHash = @{
        Path             = $this.FullName
        DestinationPath  = $target
        CompressionLevel = 'Optimal'
        Force            = $True
        PassThru         = $True
    }

    Compress-Archive @paramHash
}

$params = @{
    TypeName   = 'System.IO.FileInfo'
    MemberType = 'ScriptMethod'
    MemberName = 'Zip'
    Value      = $zip
    Force      = $True
}
Update-TypeData @params

In [ ]:
Get-ChildItem c:\temp -File | Get-Member zip

In [ ]:
Get-ChildItem c:\temp -File | Where-Object size -ge 1mb | ForEach-Object { $_.zip() }

In [ ]:
#I can use parameter since I know about it. Not easily discoverable.
Get-ChildItem c:\temp -File | Where-Object size -ge 1mb | ForEach-Object { $_.zip('c:\work') }

## Custom Formatting

- Needs a custom format file defined in a ps1xml file
- Can create multiple views of the same object

In [ ]:
#Windows PowerShell
powershell.exe -noprofile -nologo -command 'Get-ChildItem $pshome\*.format.ps1xml'

- Format files internalized in PowerShell 7
- Create your own format.ps1xml files
- https://github.com/jdhitsolutions/PSScriptTools/blob/main/docs/New-PSFormatXML.md
- Need a sample object with values for any property you want to use.

### Requirements

Your custom object must have a defined and __unique type__ name. It cannot be a generic PSCustomObject. 

```XML
<ViewSelectedBy>
    <TypeName>OBJECT.TYPE</TypeName>
</ViewSelectedBy>
```

I typically do this when creating custom objects:

```powershell
[PSCustomObject]@{
    PSTypeName   = "PSFoo"  #<--- unique type name
    Name         = $env:USERNAME
    ComputerName = $env:COMPUTERNAME
    Status       = 'Online'
    ID           = 32345
    PSVersion    = $PSVersionTable.PSVersion
}
```

PowerShell classes are defined with a type name.

```powershell
class PSFoo {
    [string]$Name
    [string]$ComputerName
    [string]$Status
    [int]$ID
    [version]$PSVersion
}
```

Or, you can insert a type name into an existing object.

```powershell
$out.PSObject.TypeNames.Insert(0,"PSFoo")
```

```powershell
$paramHash = @{
    GroupBy    = 'Directory'
    Properties = 'Name', 'Size', 'CreationTime', 'Modified', 'ModifiedAge'
    viewName   = 'AgeInfo'
    formatType = 'Table'
    Path       = '.\jhFileInfo.format.ps1xml'
}

Get-Item C:\temp\a.jpg | New-PSFormatXML @paramHash
```

In [ ]:
# I customized the file even further
code .\jhFileInfo.format.ps1xml

In [ ]:
Update-FormatData -AppendPath .\jhFileInfo.format.ps1xml

In [ ]:
dir c:\temp -file | Format-Table -view AgeInfo

## Extending Your Work

```powershell
#requires -version 7.4
#requires -module CimCmdlets
#requires -module SmbShare

#the function assumes your credential has admin rights on any remote computer
Function Get-ServerDetail {
    [cmdletbinding()]
    [OutputType('PSServerDetail')]
    Param(
        [Parameter(Position = 0, ValueFromPipeline)]
        [ValidateNotNullOrEmpty()]
        [Alias('CN')]
        [string]$Computername = $env:computername
    )

    Begin {
        Write-Verbose "Starting $($MyInvocation.MyCommand)"
    } #begin

    Process {
        #create a temporary CimSession
        Try {
            Write-Verbose "Creating a temporary CimSession for $($Computername.ToUpper())"
            $cs = New-CimSession -ComputerName $Computername -ErrorAction Stop
        }
        Catch {
            Write-Warning "Failed to create a CimSession for $($Computername.ToUpper()). $($_.Exception.Message)"
        }
        If ($cs) {
            Write-Verbose "Getting operating system information for $($Computername.ToUpper())"
            $os = Get-CimInstance -ClassName Win32_OperatingSystem -CimSession $cs
            Write-Verbose "Getting shares for $($Computername.ToUpper())"
            #ignore errors if there are no shares
            $shares = Get-SmbShare -CimSession $Computername -Special $False -ErrorAction SilentlyContinue
            #get services
            Write-Verbose "Getting running services for $($Computername.ToUpper())"
            $svc = Get-CimInstance -ClassName Win32_Service -CimSession $cs -Filter "State = 'Running'"

            Write-Verbose "Creating a custom server object for $($Computername.ToUpper())"
            [PSCustomObject]@{
                PSTypeName       = 'PSServerDetail'
                Computername     = $os.CSName
                OperatingSystem  = $os.Caption
                InstallDate      = $os.InstallDate
                Memory           = $os.TotalVisibleMemorySize
                FreeMemory       = $os.FreePhysicalMemory
                RunningProcesses = $os.NumberOfProcesses - 2  #subtract System and Idle processes
                RunningServices  = $svc.Count
                LastBoot         = $os.LastBootUpTime
                Shares           = $shares
            }
            Write-Verbose "Removing temporary CimSession for $($Computername.ToUpper())"
            Remove-CimSession -CimSession $cs
        } #If CimSession
    } #process

    End {
        Write-Verbose "Ending $($MyInvocation.MyCommand)"
    } #end

} #close function

#Extending type data
$splat = @{
    TypeName   = 'PSServerDetail'
    MemberType = 'ScriptProperty'
    MemberName = $Null
    Value      = $null
    Force      = $True
}

$splat.MemberName = 'Uptime'
$splat.Value = { New-TimeSpan -Start $this.LastBoot -End (Get-Date) }
Update-TypeData @splat

$splat.MemberType = 'NoteProperty'
$splat.MemberName = 'AuditDate'
$splat.Value = (Get-Date)
Update-TypeData @splat

$splat.MemberType = 'AliasProperty'
$splat.MemberName = 'OS'
$splat.Value = 'OperatingSystem'
Update-TypeData @splat
#load my custom format file
Update-FormatData $PSScriptRoot\PSServerDetail.format.ps1xml
```

```powershell
PS C:\> Get-ServerDetail DOM1 | Select *

Computername     : DOM1
OperatingSystem  : Microsoft Windows Server 2019 Standard Evaluation
InstallDate      : 5/16/2025 12:57:23 PM
Memory           : 2540480
FreeMemory       : 943264
RunningProcesses : 46
RunningServices  : 75
LastBoot         : 5/16/2025 1:10:25 PM
Shares           : {MSFT_SmbShare (Name = "NETLOGON", ScopeName = "*"), MSFT_SmbShare (Name = "...}
Uptime           : 12.00:37:47.2519904
AuditDate        : 5/28/2025 1:47:01 PM
OS               : Microsoft Windows Server 2019 Standard Evaluation
```

Here's an example from my test domain that better displays the custom formatting.

![Get-ServerDetail Formatting](images/get-serverdetail.png)

I think this is easier to read and more informative than the default table view.

### Alternate Views

![Memory](images/memory-view.png)

![default list view](images/serverdetail-list.png)

## Modules and Custom Formats

For stand-alone functions I typically insert this code at the end of the script.

```powershell
Update-FormatData -AppendPath $PSScriptRoot\PSServerDetail.format.ps1xml
```

For modules, I typically store format files in a subfolder. I load them in the module manifest. This is from the [PSBluesky module]( https://github.com/jdhitsolutions/PSBluesky)

```JSON
FormatsToProcess     = @(
    'formats\PSBlueSkyTimelinePost.format.ps1xml',
    'formats\PSBlueskyBlockedUser.format.ps1xml',
    'formats\PSBlueskyBlockedList.format.ps1xml',
    'formats\PSBlueskyProfile.format.ps1xml',
    'formats\PSBlueskyFollower.format.ps1xml',
    'formats\PSBlueskyFeed.format.ps1xml',
    'formats\PSBlueskyLiked.format.ps1xml',
    'formats\PSBlueskySession.format.ps1xml',
    'formats\PSBlueskyNotification.format.ps1xml',
    'formats\PSBlueskySearchResult.format.ps1xml',
    'formats\PSBlueskyModuleInfo.format.ps1xml'
)
```

The same is true for type extensions.

```powershell
TypesToProcess       = @(
    'types/PSBlueSky.types.ps1xml'
)
```

## Other Module Examples

- [PSProjectStatus](https://github.com/jdhitsolutions/PSProjectStatus/tree/main/formats)

![Get-PSProjectStatus](images/get-psprojectstatus.png)

- [AD Reporting Tools](https://github.com/jdhitsolutions/ADReportingTools/tree/main/formats)

![Get-ADDomainControllerHealth](images/get-dchealth.png)

- [PSWorkItem](https://github.com/jdhitsolutions/PSWorkItem/tree/main/formats)

![Get-PSWorkItemCategory](images/get-psworkitemcategory.png)

## Questions and Answers

### *What else do you need to know to make PowerShell do more?*





### [https://jdhitsolutions.github.io](https://jdhitsolutions.github.io)

![Photo by Ann H: https://www.pexels.com/photo/brown-wooden-letter-blocks-6732759/](images/pexels-ann-h-45017-6732759.jpg)